In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

TICKERS = ['^SPX','^IXIC','^RUT','^DJI','^XAX']
START_DATE = '1990-01-01'

In [6]:
from tqdm import tqdm

TABLE_NAMES = {
    'SP500':'^SPX',
    'NASDAQ':'^IXIC',
    'RUSSELL':'^RUT',
    'DOWJONES':'^DJI',
    'AMEX':'^XAX'
}
DB_PATH = '../../data/price.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [14]:
for table_name in tqdm(list(TABLE_NAMES.keys())) :
    data = yf.download(
        TABLE_NAMES[table_name],
        start = START_DATE,
        progress = False,
        interval = '1d'
    )
    data = data.reset_index()
    data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    
    for _, row in data.iterrows():  # Use iterrows to iterate through each row
        cursor.execute( # SQL Query Statement
            f"""INSERT OR IGNORE INTO {table_name}(
                timestamp, open, high, low, close, adj_close, volume
            )
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """, 
            (
                row['Date'].strftime('%Y-%m-%d'), 
                row['Open'], 
                row['High'], 
                row['Low'], 
                row['Close'], 
                row['Adj Close'], 
                row['Volume']
            )
        )
        conn.commit()

100%|██████████| 5/5 [00:17<00:00,  3.40s/it]
